In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import random
# random.seed(42)
import math

#to control formatting
np.set_printoptions(precision=2 , suppress= True)

: 

# Helpers


In [ ]:
#make 3D axes have equal scale to keep shapes non distorted
def equal_axes(ax) :
    #get axis limits
    x_limits = ax.get_xlim3d() 
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    print (x_limits , y_limits , z_limits)
    print ("----------------------------")

    #calculate width
    x_width = abs(x_limits[1] - x_limits[0])
    y_width = abs(y_limits[1] - y_limits[0])
    z_width = abs(z_limits[1] - z_limits[0])

    #max width to scale accordingly
    max_width = max(x_width , y_width , z_width)

    #Centers
    x_center = np.mean(x_limits)
    y_center = np.mean(y_limits)
    z_center = np.mean(z_limits)

    #new symmetric limits so all axes use the same scale
    ax.set_xlim3d(x_center - max_width/2, x_center + max_width/2)
    ax.set_ylim3d(y_center - max_width/2, y_center + max_width/2)
    ax.set_zlim3d(z_center - max_width/2, z_center + max_width/2)

#generate random point
def random_3_values(n) :
    x = random.uniform(-n,n)
    y = random.uniform(-n,n)
    z = random.uniform(-n,n)
    return (x,y,z)




# Input

In [ ]:
points = []
#fill 10 points
for i in range(10): 
    points.append(random_3_values(15))

#transforming to numpy array
points = np.array(points)
[ 0 1 2
   3 4 5
 6 7 8]
#plotting original points
fig = go.Figure(data=[go.Scatter3d(
    x=points[:,0], 
    y=points[:,1], 
    z=points[:,2],
    mode='markers', 
    textposition="top center",
    marker=dict(
        size=8,
        color=np.arange(len(points)),   #unique color 
        colorscale="Viridis",
        line=dict(width=1, color="black") # outline for visibility
    )
)])

# set axis labels and title
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube' # keep equal axes
    ),
    title="Original points"
)

fig.show()

# Movement


In [ ]:
# translation values
tx , ty , tz = random_3_values(8)

# rotation values
theta_x , theta_y , theta_z = random_3_values(math.pi * 2)

#Rotation matrices _i copied the materices
Rx = np.array([
    [1, 0, 0],
    [0, np.cos(theta_x), -np.sin(theta_x)],
    [0, np.sin(theta_x),  np.cos(theta_x)]
])

Ry = np.array([
    [ np.cos(theta_y), 0, np.sin(theta_y)],
    [ 0,               1, 0],
    [-np.sin(theta_y), 0, np.cos(theta_y)]
])

Rz = np.array([
    [np.cos(theta_z), -np.sin(theta_z), 0],
    [np.sin(theta_z),  np.cos(theta_z), 0],
    [0,                0,               1]
])


# Transformation

In [ ]:
class Transformation() :
    
    def __init__(self ,Rx , Ry , Rz , tx , ty , tz , points):
        self.Rx = Rx
        self.Ry = Ry
        self.Rz = Rz
        self.tx = tx
        self.ty = ty
        self.tz = tz
        self.points = points
        #total rotation
        self.R =  self.Rz @ self.Ry @ self.Rx  

    #----------------------
    #         HTM
    #---------------------  
    def Homogeneous_approach(self , points = None) :
        if points is None:
            points = self.points

        #build HTM
        HTM = np.eye(4)                            #start with 4x4 identity
        HTM[:3 , :3] = self.R                            #the left 3x3 is responsible for rotation
        HTM[:3 , 3] = np.array([self.tx , self.ty , self.tz])      #top right col for translation

        #Points to homo
        N = self.points.shape[0]
        #https://www.geeksforgeeks.org/python/numpy-hstack-in-python/
        points_homog = (np.hstack([self.points , np.ones((N,1))])).T #appends ones

        #apply HTM
        transferred = (HTM @ points_homog).T
        transferred_points = transferred[: , :3]    

        return transferred_points
    
    #--------------------
    #   Quaternion
    #--------------------
    def Quaternion_approach(self,points = None) :
        if points is None :
            points = self.points
        
        m = self.R
        trace = np.trace(self.R)
        
        #i'll copy this equations too T-T but i understand the concepts through researching
        if trace > 0:
            s = 0.5 / np.sqrt(trace + 1.0)
            qw = 0.25 / s
            qx = (m[2,1] - m[1,2]) * s
            qy = (m[0,2] - m[2,0]) * s
            qz = (m[1,0] - m[0,1]) * s
        else:
            if (m[0,0] > m[1,1]) and (m[0,0] > m[2,2]):
                s = 2.0 * np.sqrt(1.0 + m[0,0] - m[1,1] - m[2,2])
                qw = (m[2,1] - m[1,2]) / s
                qx = 0.25 * s
                qy = (m[0,1] + m[1,0]) / s
                qz = (m[0,2] + m[2,0]) / s
            elif m[1,1] > m[2,2]:
                s = 2.0 * np.sqrt(1.0 + m[1,1] - m[0,0] - m[2,2])
                qw = (m[0,2] - m[2,0]) / s
                qx = (m[0,1] + m[1,0]) / s
                qy = 0.25 * s
                qz = (m[1,2] + m[2,1]) / s
            else:
                s = 2.0 * np.sqrt(1.0 + m[2,2] - m[0,0] - m[1,1])
                qw = (m[1,0] - m[0,1]) / s
                qx = (m[0,2] + m[2,0]) / s
                qy = (m[1,2] + m[2,1]) / s
                qz = 0.25 * s
        
        q = np.array([qw , qx , qy,qz])

        #normalize
        q = q/np.linalg.norm(q)

        #rotation matrix back from quaternion
        Rq = np.array([
            [1 - 2*(qy**2 + qz**2),     2*(qx*qy - qz*qw),     2*(qx*qz + qy*qw)],
            [2*(qx*qy + qz*qw),         1 - 2*(qx**2 + qz**2), 2*(qy*qz - qx*qw)],
            [2*(qx*qz - qy*qw),         2*(qy*qz + qx*qw),     1 - 2*(qx**2 + qy**2)]
        ])

        #apply quaternion
        transformed_points = (Rq @ points.T).T + np.array([self.tx, self.ty, self.tz])

        return transformed_points

    # Method to decide if HTM or quaternion to use
    def transform(self) :
        #gimble lock
        if np.isclose(abs(theta_y) , np.pi/2 , atol= 1e-3) :
            print("Gimbal lock detected, using Quaternion")
            return self.Quaternion_approach()
        else :
            return self.Homogeneous_approach()







# Plotting

In [ ]:
import plotly.graph_objects as go
import numpy as np

def plot_points_and_frames(points, transformed_points):
    """
    Visualize original vs transformed point pairs and coordinate frames.

    Args:
        points (ndarray): Nx3 array of original points
        transformed_points (ndarray): Nx3 array of transformed points
    """
    # derive approximate HTM from the point sets
    # here: compute R, t using Procrustes (assuming rigid transform)
    origin = np.array([0,0,0])
    axes = np.eye(3)  # unit X, Y, Z

    # estimate translation as mean difference
    t = transformed_points.mean(axis=0) - points.mean(axis=0)

    # align covariance
    H = (points - points.mean(axis=0)).T @ (transformed_points - transformed_points.mean(axis=0))
    U, _, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:  # ensure proper rotation
        Vt[-1,:] *= -1
        R = Vt.T @ U.T

    # transformed axes
    axes_trans = (R @ axes.T).T

    colors = ["red", "green", "blue"]  # for axes

    # generate a colormap for point pairs
    pair_colors = [f"rgb({r},{g},{b})" for r,g,b in np.random.randint(0,255,(len(points),3))]

    fig = go.Figure()

    # plot each pair of points (original + transformed + line)
    for i, c in enumerate(pair_colors):
        # original
        fig.add_trace(go.Scatter3d(
            x=[points[i,0]], y=[points[i,1]], z=[points[i,2]],
            mode="markers",
            marker=dict(size=6, color=c, symbol="circle"),
            name="original" if i==0 else None,
            showlegend=(i==0)
        ))

        # transformed
        fig.add_trace(go.Scatter3d(
            x=[transformed_points[i,0]], y=[transformed_points[i,1]], z=[transformed_points[i,2]],
            mode="markers",
            marker=dict(size=6, color=c, symbol="square"),
            name="transformed" if i==0 else None,
            showlegend=(i==0)
        ))

        # line between them
        fig.add_trace(go.Scatter3d(
            x=[points[i,0], transformed_points[i,0]],
            y=[points[i,1], transformed_points[i,1]],
            z=[points[i,2], transformed_points[i,2]],
            mode="lines",
            line=dict(color=c, width=2),
            showlegend=False
        ))

    # original frame (solid)
    for i, c in enumerate(colors):
        fig.add_trace(go.Scatter3d(
            x=[origin[0], axes[i,0]],
            y=[origin[1], axes[i,1]],
            z=[origin[2], axes[i,2]],
            mode="lines",
            line=dict(color=c, width=6),
            name=f"original {['X','Y','Z'][i]}"
        ))

    # transformed frame (dashed, shifted by t)
    for i, c in enumerate(colors):
        fig.add_trace(go.Scatter3d(
            x=[t[0], t[0]+axes_trans[i,0]],
            y=[t[1], t[1]+axes_trans[i,1]],
            z=[t[2], t[2]+axes_trans[i,2]],
            mode="lines",
            line=dict(color=c, width=3, dash="dash"),
            name=f"transformed {['X','Y','Z'][i]}"
        ))

    # layout
    fig.update_layout(
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z"
        ),
        title="Original vs Transformed Point Pairs + Frames"
    )

    fig.show()

In [ ]:
trans = Transformation(Rx, Ry, Rz, tx, ty, tz, points)
transformed_points = trans.transform()
plot_points_and_frames(points, transformed_points)
